In [1]:
file_number = 4
with open("../data/rawData/"+ str(file_number)+".txt","r") as f:
    global data
    data=f.readlines()
# print(data)
node_number = int(data[3])

In [2]:
types = data[-1].strip().split(' ')

# 按照顺序后的ip核排列
def seqList(types):
    type_set = set()
    typeSetSeq = []
    for i in types:
        if i not in type_set:
            type_set.add(i)
            typeSetSeq.append(i)
    return typeSetSeq

typeSetSeq = seqList(types)  

In [3]:
typeSetSeq # 按照顺序后的ip核排列

['1', '0']

In [4]:
# 输入n个受信任ip核
while(True):
    n_ip= int(input("请输入n个受信任ip核:"))
    if n_ip<=len(typeSetSeq):
        break;

请输入n个受信任ip核:0


In [5]:
# 建立矩阵
'''
数据文件：2.txt内容：（以空格分开每个数据）
1 2 2.5
3 4 4
7 8 7
'''
 
from numpy import *
raw_matrix = zeros((node_number,node_number),dtype=int)    
# print(raw_matrix)
# 获取所有的矩阵行
lines = data[5:-2]
# print(lines)
raw_matrix_row = 0                       #表示矩阵的行，从0行开始
for line in lines:              #把lines中的数据逐行读取出来
    list = line.strip().strip('\n').split(' ')      #处理逐行数据：strip表示把头尾的'\n'去掉，split表示以空格来分割行数据，然后把处理后的行数据返回到list列表中
#     print(list)
    raw_matrix[raw_matrix_row:] = list[:node_number]                    #把处理后的数据放到方阵A中。list[0:3]表示列表的0,1,2列数据放到矩阵A中的A_row行
    raw_matrix_row+=1                                #然后方阵A的下一行接着读
    #print(line)

# print(raw_matrix)    #打印 方阵A里的数据
 

In [7]:
# 转换为对称矩阵
def toNoDirectGraph(matrix,n):
    for i in range(n):
        for j in range(n):
            if matrix[i][j]==1:
                matrix[j][i]=matrix[i][j]
#             elif matrix[j][i]==1:
#                 matrix[i][j]=matrix[j][i]
    return matrix


In [8]:
def computeParentNode(matrix,n):
    raw = matrix
    for i in range(n):
        for j in range(i+1,n):
            for k in range(n):
                if matrix[i][k]==1 and matrix[j][k]==1:
                    raw[i][j]=1
    return raw
# print(raw_matrix)
# print(computeParentNode(raw_matrix,node_number))
# print(raw_matrix)

In [9]:
# 矩阵扩充
def generateMatrix(matrix,n):
    actual_number=n*3
    bigMatrix = zeros((actual_number,actual_number),dtype=int) 
    for i in range(actual_number):
        for j in range(actual_number):
            # 开始填充左上角部分
            if i<n and j<n:
                bigMatrix[i][j]=matrix[i][j]
                # 连接特殊关系
                bigMatrix[i][i+n]=1
                bigMatrix[i][i+2*n]=1
            if i==0 and j<n:    
                bigMatrix[j+n][j+2*n]=1
            # 开始扩充中间部分
            if (i>=n and i<2*n) and (j>=n and j<2*n):
                bigMatrix[i][j]=matrix[i-n][j-n]
            # 开始扩充右下角部分
            if (i>=2*n and i<3*n) and (j>=2*n and j<3*n):
                bigMatrix[i][j]=matrix[i-2*n][j-2*n]
    return bigMatrix

In [10]:
bigMatrix=generateMatrix(raw_matrix,node_number)
print(bigMatrix)

[[0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]]


In [11]:
bigMatrix = toNoDirectGraph(bigMatrix,3*node_number) 
print(bigMatrix)
print(bigMatrix.shape)

[[0 1 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 1 0 1]
 [0 0 0 ... 0 1 0]]
(138, 138)


In [12]:
# time space is O(n)
def getLoc(types,seq):
    # 从types列表中得到受信任ip核的index(实际位置-1)
    pos = []
    for i in range(len(types)):
        # set查找O(1)
        if types[i] in seq:
            pos.append(i)
    return pos


# n如果等于typeSetSeq长度，表示全部ip核受信任,直接计算vendor1的价格,直接调到后面执行vendor计算

# 否则接着往下执行

pos=[]
# 有些是受信任,有些是不受信任的,剔除受信任的ip核
if n_ip>0 and n_ip<len(typeSetSeq):
    frontSeq  = typeSetSeq[:n_ip]  # 受信任的ip核
    print(frontSeq)
    pos = getLoc(types,frontSeq)   # 受信任的ip核对应的index
    print(pos)
    # 扩充pos到三倍长度
    pos_1 = [i+node_number for i in pos]
    print(pos_1)
    pos_2 = [i+2*node_number for i in pos]
    print(pos_2)
    pos = pos+pos_1+pos_2
    print(pos)   # 扩充到三倍后的ip核对应的index
        # 修改bigMatrix
    for i in range(3*node_number):
        if i in pos:
            for j in range(3*node_number):
                bigMatrix[i][j]=0
        else:
            for j in range(3*node_number):
                if j in pos:
                    bigMatrix[i][j]=0
    bigMatrix

In [13]:
bigMatrix

array([[0, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 1, 0, 1],
       [0, 0, 0, ..., 0, 1, 0]])

In [14]:
# 算法1:图着色实现
count = 0
big_n = 3*node_number
color=[0]*(big_n+1)
result_list=[]

def ok(k,matrix):
    for i in range(1,k):
        if matrix[k-1][i-1]==1 and color[i]==color[k]:
            return 0
    return 1
def graphcolor(n,m,matrix):
    for i in range(1,n+1):
        color[i]=0
    k=1
    while k>=1:
        color[k]=color[k]+1;
        while color[k]<=m:
            if ok(k,matrix):
                break
            else:
                color[k]=color[k]+1 #搜索下一个颜色
        if color[k]<=m and k==n:
            for i in range(1,n+1):
                print("第"+ str(i) +"个结点着第"+ str(color[i]) +"色!\n");
                result_list.append((color[i],i))
            return 1;
        elif color[k]<=m and k<n:
            k=k+1 #处理下一个顶点
        else:
            color[k]=0;
            k=k-1 #回溯
    return 0  
leastpoint = 1
res = 0
while 1:
    print(leastpoint)
    res = graphcolor(big_n,leastpoint,bigMatrix) #从1开始
    if res:
        print("该矩阵至少需要"+ str(leastpoint) +"种颜色着色!\n")
        break
    leastpoint+=1
    
# print(result_list)
result_list.sort()
# print(result_list)
result_dict={}
each_list=[]
result_list.append((0,0))

for i in range(len(result_list)-1):
    each_list.append(result_list[i][1])
    if result_list[i][0]!=result_list[i+1][0]:
        result_dict[result_list[i][0]]=each_list
        each_list=[]
result_list.pop()
print(result_list)
print(result_dict)

1
2
3


KeyboardInterrupt: 

In [ ]:
import copy
save_dict = copy.deepcopy(result_dict)
max_len = 0
for k,v in save_dict.items():
    max_len=max(max_len,len(v))
for k,v in save_dict.items():
    if len(v)<max_len:
        for i in range(max_len-len(v)):
            v.append(-1)
save_dict

In [15]:
# result_dict[1]=[1,4,7,8,15]
# result_dict[2]=[2,3,10,11,14]
# result_dict[3]=[5,6,9,12,13]
# result_dict
import pandas as pd
save = pd.DataFrame(save_dict)

save.to_csv('../data/graphColor/'+str(file_number)+'_'+str(n_ip)+'_cipr.txt',sep=' ',index=False)

In [16]:
# 将D集合的对应节点转化为运算符
# 建立节点与IP核映射关系
types_list=(data[-1].strip().split(' '))*3
print(types_list)
# types
node_list = []
for i in range(1,3*node_number+1):
    node_list.append(i)
print(node_list)
node_ip = dict(zip(node_list,types_list))
print(node_ip)
# 开始转换
for key in result_dict:
    each_list = []
    for each in result_dict[key]:
        each_list.append(node_ip[each])
    result_dict[key]=each_list
    
print(result_dict)

['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87]
{1: '0', 2: '0', 3: '0', 4: '0', 5: '0', 6: '0', 7: '0', 8: '0', 9: '0', 10: '0', 11: '0', 12: '0', 13: '0', 14: '0', 15: '0', 16: '0', 17: '0', 18: '0', 19: '0', 20: '0', 21: '0', 22: '0', 23: '0', 24: '0', 25: '0', 26: '0'

In [17]:
save_dict = copy.deepcopy(result_dict)
max_len = 0
for k,v in save_dict.items():
    max_len=max(max_len,len(v))
for k,v in save_dict.items():
    if len(v)<max_len:
        for i in range(max_len-len(v)):
            v.append(-1)
save_dict

save = pd.DataFrame(save_dict)

save.to_csv('../data/graphColor/'+str(file_number)+'_'+str(n_ip)+'_cip.txt',sep=' ',index=False)

In [18]:
result_dict

{1: ['0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0'],
 2: ['0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0'],
 3: ['0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0']}

In [19]:
#生成权重矩阵
# 读取厂商提供的权重数据
import pandas as pd
vendor_weight = pd.read_csv('../data/rawData/weight.csv')
# print(vendor_weight)
# 开始计算权重

s,col = vendor_weight.shape

# print(s)

weightMatrix = zeros((s,leastpoint),dtype=int) # leastpoint=m 最少着色数

# print(weightMatrix)

print(vendor_weight)

# print(vendor_weight['0'])  #  第'0'列

# 全部是受信任的ip核，直接按照vendor1价格计算
if n_ip==len(typeSetSeq):
    totalprice=0
    for each in typeSetSeq:
        totalprice += vendor_weight[each][0] # 列,行  按照vendor1的价格计算
    print(totalprice)
# 有受信任的ip核,按照vendor1价格计算
elif n_ip>0 and n_ip<len(typeSetSeq):
    frontSeq  = typeSetSeq[:n_ip]  # 受信任的ip核
    print(frontSeq)
    # 考虑vendor1的价格作为受信任ip核的价格
    for key in result_dict: #颜色填充列
        i=0 
        while i<s:  # 每一行
            totalprice = 0
            d_set = set()
            # 颜色:对应节点对应的ip核
            for each in result_dict[key]:
                if each in d_set:
                    continue
                else:
                    d_set.add(each)
                    if each in frontSeq:  # 受信任ip核
                        totalprice += vendor_weight[each][0] # 列,行  按照vendor1的价格计算
                    else: # 是不受信任的ip核心
                        totalprice += vendor_weight[each][i] # 列,行  
            weightMatrix[i][key-1]=totalprice
            i+=1
    print(-weightMatrix.T)
else:
    # 都是不受信任的ip核计算
    for key in result_dict: #颜色填充列
        i=0 
        while i<s:  # 每一行
            totalprice = 0
            d_set = set()
            # 颜色:对应节点对应的ip核
            for each in result_dict[key]:
    #             print(each)
                if each in d_set:
                    continue
                else:
                    d_set.add(each)
                    totalprice += vendor_weight[each][i] # 列,行
            weightMatrix[i][key-1]=totalprice
            i+=1
    print(-weightMatrix.T)

   vendor    0     1    2    3
0       1  539  1128  877  393
1       2  548  1192  863  354
2       3  585  1111  886  311
3       4  565  1127  894  385
4       5  535  1124  849  378
5       6  581  1149  882  315
[[-539 -548 -585 -565 -535 -581]
 [-539 -548 -585 -565 -535 -581]
 [-539 -548 -585 -565 -535 -581]]


In [20]:
# print(weightMatrix.shape)
# row,col=weightMatrix.shape
# match_result = []
# row_set = set()
# for r in range(col):
#     min = 0
#     bestrow=1
#     bestcol=r+1
#     for c in range(row):
#         if weightMatrix[c][r]<min:
#             min=weightMatrix[c][r]
#             if bestrow not in row_set:
#                 row_set.add(bestrow)
#             else:
#             bestrow=c+1
            
#     match_result.append((bestrow,bestcol))

# print(match_result)

In [21]:
import numpy as np
np.savetxt("../data/graphColor/"+str(file_number)+'_'+str(n_ip)+'_weight.txt',-weightMatrix.T,fmt="%d")